# The finite difference time domain method (FDTD) in two dimensions

### Electromagnetiv scattering problems

When an electromagnetic field encounters a target, currents are excited on it, which in turn re-radiate. This process is called “electromagnetic scattering”. Since the Maxwell equations are linear the fields are often decomposed into an _incident_ field $E^{inc}$ and a _scattered_ field $E^{scat}$. The overall field, called the _total_ field $E^{tot}$, is then:
$$E^{tot} = E^{inc}+E^{scat}$$
By definition, the incident field is the field which would exist if the scatterer were absent.

<div style="align: left; text-align:center;">
  <img src="Fig3_2_Davidson.png" alt="Drawing" style="width: 400px;">
  <figcaption> <strong>Fig. 1</strong> The scatterer and surrounding FDTD zones, showing scattered field and total field regions </figcaption>
</div>

### Including a source: the scattered/total field formulation

If we want to study scattering, we need a method for simulating a plane wave. The simplest method for doing this is to exploit the concepts of incident, scattered and total fields, introduced in the section above. Since the Maxwell equations are linear we can use the FDTD to solve for either the scattered or total fields. We will split the computational area into two zones using a (non-physical) line, as in Fig. 1. 
In one region, we will have only scattered fields, and in the other, total fields. For convenience, we can choose a constant $x$ coordinate (this corresponds to the variable `L` in the simulation code).

Since three of the five field components in the two-dimensional Yee cell are located at half-step
values, our scattered/total field demarcation is located at
$x_L = (L - \frac{1}{2})\Delta$. Fields located _on_ and to the right of this line this will be chosen as
total fields. Fields to the left will be scattered fields.

### Historical Background: The Berenger´s split field formulation

The Berenger´s split field formulation is the simplest and most efficient approach in two dimensions. The reason for this is that, contrary to the (Uniaxial) UPML, we can avoid the storage requirements of electric and magnetic flux vectors and the additional complexity due to dealing with dispersive materials. It is important to note that this benefit accrues only in two dimensions.

Berenger recognized that an additional degree of freedom would permit a match off normal as well. He did this by splitting the transverse fields into two orthogonal components, for example $H_z = H_{zx} + H_{zy}$. Applying this to a two-dimensional TE problem described by the Maxwell-equations with lossless materials _(Eq: 3.5 - 3.7)_:

$$\frac{\partial E_x}{\partial t} = \frac{1}{\epsilon}\frac{\partial H_z}{\partial y}$$

$$\frac{\partial E_y}{\partial t} = -\frac{1}{\epsilon}\frac{\partial H_z}{\partial x}$$

$$\frac{\partial H_z}{\partial t} = \frac{1}{\mu} \biggl(\frac{\partial E_x}{\partial y} - \frac{\partial E_y}{\partial x} \biggl)$$

result in four equations:

$$\frac{\partial E_x}{\partial t} = \frac{1}{\epsilon} \frac{\partial (H_{zx} + H_{zy})}{\partial y}$$

$$\frac{\partial E_y}{\partial t} = -\frac{1}{\epsilon} \frac{\partial (H_{zx} + H_{zy})}{\partial x}$$

$$\frac{\partial H_{zx}}{\partial t} = -\frac{1}{\mu}\frac{\partial E_y}{\partial x}$$

$$\frac{\partial H_{zy}}{\partial t} = \frac{1}{\mu}\frac{\partial E_x}{\partial y}$$

### Perfectly matched Layer

The perfectly matched layer (PML) is the theoratical approach for the boundary in sumulating an open field. The "real" open field would not have any boundary. With the FDTD we need a boundary conditions to be able to do a simulation. The perfect boundary for the simluation would absorbe all incoming waves. The PML is an effective approach to simulate such an absorbing boundary.

### The FDTD update equations for a PML

With the theoretical background, associated with two components of $\sigma^*$ ($\sigma_x^*$ and $\sigma_y^*$) and two components of $\sigma$ ($\sigma_x$ and $\sigma_y$), describing the electrical ($\sigma$) and magnetic ($\sigma^*$) loss, the time domain equations are _(Eq.: 3.58 - 3.61)_:

$$ \biggl( \epsilon \frac{\partial}{\partial t}+\sigma_y \biggl) E_x = \frac{\partial(H_{zx}+H_{zy})}{\partial y}$$

$$ \biggl( \epsilon \frac{\partial}{\partial t}+\sigma_x \biggl) E_x = -\frac{\partial(H_{zx}+H_{zy})}{\partial x}$$

$$ \biggl( \mu \frac{\partial}{\partial t}+\sigma_x^* \biggl) H_zx = -\frac{\partial E_y}{\partial x}$$

$$ \biggl( \mu \frac{\partial}{\partial t}+\sigma_y^* \biggl) H_zy = \frac{\partial E_x}{\partial y}$$

To evaluate the electric field, we use the _semi-implicit_ approximation. It is needed, because of the problem of evaluating the electric field at a half time step. The required value is computed as the average of the previous (known) value and the as-yet-to-be-computed value:

$$E_x\biggl(i+\frac{1}{2},j,n+\frac{1}{2}\biggl)= \frac{E_x(i+\frac{1}{2},j,n+1)+E_x(i+\frac{1}{2},j,n)}{2}$$

Using this approximation, and otherwise proceeding as before, the result is the following set of update equations _(Eq.: 3.63 - 3.66)_:

$$H_{zx}(i,j,n)=D_{a_{Hzx}}(i,j)\cdot H_{zx}(i,j,n-1)-D_{b_{Hzx}}(i,j)\cdot [E_y(i+1,j,n) - E_y(i,j,n)]$$

$$H_{zy}(i,j,n)=D_{a_{Hzy}}(i,j)\cdot H_{zy}(i,j,n-1)-D_{b_{Hzy}}(i,j)\cdot [E_x(i,j+1,n) - E_x(i,j,n)]$$

$$E_x(i,j,n+1)=C_{a_{Ex}}(i,j)\cdot E_{x}(i,j,n)+C_{b_{Ex}}(i,j)\cdot [H_z(i,j,n) - H_z(i,j-1,n)]$$

$$E_y(i,j,n+1)=C_{a_{Ey}}(i,j)\cdot E_y(i,j,n)-C_{b_{Ey}}(i,j)\cdot [H_z(i,j,n) - H_z(i-1,j,n)]$$

, where we have combined the $H$ field _(Eq.:3.67)_:

$$H_z(i,j,n) = H_{zx}(i,j,n)+H_{zy}(i,j,n)$$

and the material constants as _(Eq.:3.68 - 3.75)_:

$$C_{a_{Ex}} (i,j) = \frac{1 - \frac{\sigma_y(i,j)\Delta t}{2\epsilon(i,j)}}{1+ \frac{\sigma_y(i,j)\Delta t}{2 \epsilon (i,j)}} \qquad \qquad \qquad 
C_{b_{Ex}} (i,j) = \frac{\frac{\Delta t}{\epsilon(i,j) \Delta y}}{1+ \frac{\sigma_y(i,j)\Delta t}{2 \epsilon (i,j)}}$$

$$C_{a_{Ey}} (i,j) = \frac{1 - \frac{\sigma_x(i,j)\Delta t}{2\epsilon(i,j)}}{1+ \frac{\sigma_x(i,j)\Delta t}{2 \epsilon (i,j)}} \qquad \qquad \qquad
C_{b_{Ey}} (i,j) = \frac{\frac{\Delta t}{\epsilon(i,j) \Delta x}}{1+ \frac{\sigma_x(i,j)\Delta t}{2 \epsilon (i,j)}}$$

$$D_{a_{Hzx}}(i,j) = \frac{1 - \frac{\sigma_x^*(i,j)\Delta t}{2\mu(i,j)}}{1+ \frac{\sigma_x^*(i,j)\Delta t}{2 \mu (i,j)}} \qquad \qquad \qquad
D_{b_{Hzx}} (i,j) = \frac{\frac{\Delta t}{\mu (i,j) \Delta x}}{1+ \frac{\sigma_x^*(i,j)\Delta t}{2 \mu (i,j)}}$$

$$D_{a_{Hzy}} (i,j) = \frac{1 - \frac{\sigma_y^*(i,j)\Delta t}{2\mu (i,j)}}{1+ \frac{\sigma_x^*(i,j)\Delta t}{2 \mu (i,j)}} \qquad \qquad \qquad
D_{b_{Hzy}} (i,j) = \frac{\frac{\Delta t}{\mu(i,j) \Delta y}}{1+ \frac{\sigma_y^*(i,j)\Delta t}{2 \mu (i,j)}}$$

## Simulation

The code allows to simulate different scenarios. The simulation region is defined by the the steps and the spatial distance of each step. In this region we can place a line source, plane wave and a PEC cylinder and/or PEC cuboid. The choosen area will be plottet later on.

The line source and the plane wave is a normalized Gaussian derivative pulse:
$$v_0(t) = - \frac{e^{1/2}}{\sigma}(t-m)e^{-(t-m)^2/2\sigma^2}$$
The normalizing constant $e^{1/2}/\sigma$ provides a unit peak amplitude at $t - m = ± \sigma$.